# Analisando ações de empresas do Brasil

Existem alguns indicares que demonstram se as ações são boas para investimentos.  
Não existe  uma biografica ou referência que possa trazer uma garantia quanto ao retorno do investimento, uma vez que se trata de investimento de risco e de alta volatibilidade.  
Por este motivo este trabalho busca unicamente apresentar um pouco das ferramentas utilizadas em Python e não busca fazer recomendações para compra de ações. 

Importando as bibliotecas necessárias para trabalhar.

In [17]:
import pandas as pd
from urllib.request import urlopen
from plotly.offline import iplot
import cufflinks as cf

cf.go_offline()

Para obter os dados foi utilizado um scraping no site Fundamentus (www.fundamentus.com.br) para capturar todas as ações listadas na bolsa de valores brasileira.  
É possível ir direto no site e baixar. 

In [2]:
html = urlopen('http://www.fundamentus.com.br/resultado.php')
arq = pd.read_html(html)
df = pd.DataFrame(arq[0])

Vamos analisar os dados obtidos.

In [3]:
df.head()

,Papel,Cotação,P/L,P/VP,PSR,Div.Yield,P/Ativo,P/Cap.Giro,P/EBIT,P/Ativ Circ.Liq,...,EV/EBITDA,Mrg Ebit,Mrg. Líq.,Liq. Corr.,ROIC,ROE,Liq.2meses,Patrim. Líq,Dív.Brut/ Patrim.,Cresc. Rec.5a
0,VNET3,0.0,0.0,0.0,0.0,"0,00%",0,0.0,0.0,0.0,...,0.0,"0,00%","0,00%",0,"0,00%","12,99%",000,9.257.250.00000,0.0,"-2,71%"
1,CFLU4,1.0,0.0,0.0,0.0,"0,00%",0,0.0,0.0,0.0,...,0.0,"8,88%","10,72%",110,"17,68%","32,15%",000,60.351.00000,6.0,"8,14%"
2,CLAN3,0.0,0.0,0.0,0.0,"0,00%",0,0.0,0.0,0.0,...,0.0,"0,00%","0,00%",0,"0,00%","-1,05%",000,1.012.240.00000,0.0,"-63,96%"
3,CLSC6,0.0,0.0,0.0,0.0,"0,00%",0,0.0,0.0,0.0,...,0.0,"0,00%","0,00%",0,"0,00%","20,15%",000,1.407.120.00000,0.0,"5,52%"
4,CSTB3,15000.0,0.0,0.0,0.0,"0,00%",0,0.0,0.0,0.0,...,0.0,"40,85%","28,98%",260,"22,40%","20,11%",000,8.420.670.00000,14.0,"31,91%"


Tive que fazer um tratamento nos dados para aplicar os filtros.

In [4]:
for x in df:
    if df[x].dtype == object:
        df[x] = df[x].str.replace('%','')
        df[x] = df[x].str.replace('.', '')
        df[x] = df[x].str.replace(',', '.')
    if x != 'Papel':    
        df[x] = df[x].astype(float)

ac = ['Cotação', 'P/L', 'P/VP', 'P/Cap.Giro', 'P/EBIT', 'P/Ativ Circ.Liq', 'EV/EBIT', 
       'EV/EBITDA', 'Liq. Corr.', 'Dív.Brut/ Patrim.']   
for x in ac:
    df[x] = df[x].astype(float) / 100

df['P/Ativo'] = df['P/Ativo'].astype(float) / 1000
df['PSR'] = df['PSR'].astype(float) / 1000

Com os dados capturados e tratados vamos utilizar um pouco da análise fundamentalista para classificar as melhores ações na Bolsa de Valores do Brasil.  

#### Valor Justo no último ano  

O valor justo é o valor que a ação vale em relação ao pagamento de dividendos anual.  
Segundo Décio Bazin em seu livro Faça Fortuna com Ações para se obter um valor justo de uma ação com pagamento de dividendos de 6% anualmente deve-se mutiplicar o valor total do dividento por 16,67.

Como o site traz o valor do dividendo em porcentagem fiz a conversão e multipliquei por 16,67.

In [5]:
df['Preco Justo'] = ((df['Cotação'] / 100) * df['Div.Yield']) * 16.67 
df['Preco Justo'] = df['Preco Justo'].round(2)

In [6]:
# Apagar essa parte do codigo.

df = df[['Papel', 'Cotação', 'Preco Justo', 'Div.Yield', 'P/L', 'P/VP', 'PSR', 'P/Ativo',
       'P/Cap.Giro', 'P/EBIT', 'P/Ativ Circ.Liq', 'EV/EBIT', 'EV/EBITDA',
       'Mrg Ebit', 'Mrg. Líq.', 'Liq. Corr.', 'ROIC', 'ROE', 'Liq.2meses',
       'Patrim. Líq', 'Dív.Brut/ Patrim.', 'Cresc. Rec.5a']]

### Roe - Retorno sobre o patrimonio líquido  

O Roe mede a capacidade de agregar valor de uma empresa à partir de seus próprios recursos e do dinheiro de investidores. 
Quanto maior o ROE melhor.

ROE = Lucro Líquido + Patrimônio Líquido

In [7]:
df_roe = df[df['ROE'] > 0]
df_roe

,Papel,Cotação,Preco Justo,Div.Yield,P/L,P/VP,PSR,P/Ativo,P/Cap.Giro,P/EBIT,...,EV/EBITDA,Mrg Ebit,Mrg. Líq.,Liq. Corr.,ROIC,ROE,Liq.2meses,Patrim. Líq,Dív.Brut/ Patrim.,Cresc. Rec.5a
0,VNET3,0.00,0.00,0.00,0.000000,0.00,0.000,0.000,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,12.99,0.0,9.257250e+11,0.00,-2.71
1,CFLU4,0.01,0.00,0.00,0.000000,0.00,0.000,0.000,0.00,0.00,...,0.00,8.88,10.72,1.10,17.68,32.15,0.0,6.035100e+09,0.06,8.14
3,CLSC6,0.00,0.00,0.00,0.000000,0.00,0.000,0.000,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,20.15,0.0,1.407120e+11,0.00,5.52
4,CSTB3,150.00,0.00,0.00,0.000000,0.00,0.000,0.000,0.00,0.00,...,0.00,40.85,28.98,2.60,22.40,20.11,0.0,8.420670e+11,0.14,31.91
5,CSTB4,147.69,0.00,0.00,0.000000,0.00,0.000,0.000,0.00,0.00,...,0.00,40.85,28.98,2.60,22.40,20.11,0.0,8.420670e+11,0.14,31.91
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
881,UBBR11,14.75,0.00,0.00,0.012018,3.91,0.000,0.000,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.33,0.0,1.031720e+12,0.00,10.58
882,UBBR3,18.00,0.00,0.00,0.014666,4.77,0.000,0.000,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.33,0.0,1.031720e+12,0.00,10.58
883,CEPE6,19.30,11.55,3.59,0.014917,0.86,-17.684,0.161,-9.37,68.89,...,254.51,-25.67,-1.19,0.95,0.27,0.06,55024.0,1.667000e+11,3.36,8.63
884,CEPE3,24.53,0.00,0.00,0.018960,1.10,-22.477,0.204,-11.91,87.55,...,269.96,-25.67,-1.19,0.95,0.27,0.06,0.0,1.667000e+11,3.36,8.63


# P/VPA

O P/VPA é um indicador financeiro que apresenta quanto o mercado está pagando por ação P(preço), sobre o VPA(valor patrimonial por ação).  
O número do resultado é interpretado como multiplicativo, e sempre indica quantas vezes estará pagando, no preço de uma ação pelo valor patrimonial.  
Alguns analista indicam um **valor abaixo ou igual a 1,5** com um bom P/VPA.  
VPA = PL/N° Ações  
Preço/VPA

In [13]:
df_vpa = df_roe[(df_roe['P/VP'] <= 2) & (df_roe['P/VP'] >= 0)]
df_vpa

,Papel,Cotação,Preco Justo,Div.Yield,P/L,P/VP,PSR,P/Ativo,P/Cap.Giro,P/EBIT,...,EV/EBITDA,Mrg Ebit,Mrg. Líq.,Liq. Corr.,ROIC,ROE,Liq.2meses,Patrim. Líq,Dív.Brut/ Patrim.,Cresc. Rec.5a
0,VNET3,0.00,0.00,0.00,0.000000,0.00,0.000,0.000,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,12.99,0.0,9.257250e+11,0.00,-2.71
1,CFLU4,0.01,0.00,0.00,0.000000,0.00,0.000,0.000,0.00,0.00,...,0.00,8.88,10.72,1.10,17.68,32.15,0.0,6.035100e+09,0.06,8.14
3,CLSC6,0.00,0.00,0.00,0.000000,0.00,0.000,0.000,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,20.15,0.0,1.407120e+11,0.00,5.52
4,CSTB3,150.00,0.00,0.00,0.000000,0.00,0.000,0.000,0.00,0.00,...,0.00,40.85,28.98,2.60,22.40,20.11,0.0,8.420670e+11,0.14,31.91
5,CSTB4,147.69,0.00,0.00,0.000000,0.00,0.000,0.000,0.00,0.00,...,0.00,40.85,28.98,2.60,22.40,20.11,0.0,8.420670e+11,0.14,31.91
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
880,GBIO33,9.92,0.00,0.00,660.430000,1.43,1.423,0.848,7.17,8.94,...,10.14,15.92,0.22,1.36,11.87,0.22,193530000.0,7.405640e+10,0.26,-1.95
881,UBBR11,14.75,0.00,0.00,0.012018,3.91,0.000,0.000,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.33,0.0,1.031720e+12,0.00,10.58
883,CEPE6,19.30,11.55,3.59,0.014917,0.86,-17.684,0.161,-9.37,68.89,...,254.51,-25.67,-1.19,0.95,0.27,0.06,55024.0,1.667000e+11,3.36,8.63
884,CEPE3,24.53,0.00,0.00,0.018960,1.10,-22.477,0.204,-11.91,87.55,...,269.96,-25.67,-1.19,0.95,0.27,0.06,0.0,1.667000e+11,3.36,8.63


# P/L

P/L - Preço sobre lucro.  
O P/L indica o **número de anos** que um acionista levaria para reaver o capital aplicado em determinada ação.
É um indicador para saber o quanto essa ação pode agregar de valor para o investidor.  
Ele indica o preço de cotação da ação divido elo LPA (lucro do últimos 12 meses) pelo número de ações total.  
P/L = P/LPA

In [14]:
df_pl = df_vpa[df_vpa['P/L'] < 15]
df_pl

,Papel,Cotação,Preco Justo,Div.Yield,P/L,P/VP,PSR,P/Ativo,P/Cap.Giro,P/EBIT,...,EV/EBITDA,Mrg Ebit,Mrg. Líq.,Liq. Corr.,ROIC,ROE,Liq.2meses,Patrim. Líq,Dív.Brut/ Patrim.,Cresc. Rec.5a
0,VNET3,0.00,0.00,0.00,0.000000,0.00,0.000,0.000,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,12.99,0.0,9.257250e+11,0.00,-2.71
1,CFLU4,0.01,0.00,0.00,0.000000,0.00,0.000,0.000,0.00,0.00,...,0.00,8.88,10.72,1.10,17.68,32.15,0.0,6.035100e+09,0.06,8.14
3,CLSC6,0.00,0.00,0.00,0.000000,0.00,0.000,0.000,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,20.15,0.0,1.407120e+11,0.00,5.52
4,CSTB3,150.00,0.00,0.00,0.000000,0.00,0.000,0.000,0.00,0.00,...,0.00,40.85,28.98,2.60,22.40,20.11,0.0,8.420670e+11,0.14,31.91
5,CSTB4,147.69,0.00,0.00,0.000000,0.00,0.000,0.000,0.00,0.00,...,0.00,40.85,28.98,2.60,22.40,20.11,0.0,8.420670e+11,0.14,31.91
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
669,TCOC3,38.85,0.00,0.00,14.900000,1.80,2.225,1.307,3.78,14.48,...,12.23,15.37,14.93,2.53,13.98,12.06,0.0,2.812920e+11,0.04,16.71
881,UBBR11,14.75,0.00,0.00,0.012018,3.91,0.000,0.000,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.33,0.0,1.031720e+12,0.00,10.58
883,CEPE6,19.30,11.55,3.59,0.014917,0.86,-17.684,0.161,-9.37,68.89,...,254.51,-25.67,-1.19,0.95,0.27,0.06,55024.0,1.667000e+11,3.36,8.63
884,CEPE3,24.53,0.00,0.00,0.018960,1.10,-22.477,0.204,-11.91,87.55,...,269.96,-25.67,-1.19,0.95,0.27,0.06,0.0,1.667000e+11,3.36,8.63


# Margem Líquida  

Margem líquida é igual ao lucro líquido que a empresa faz num período dividido pela receita líquida.    
A cada real da receita operacional líquida obtido depois de pagar todas as despesas operacionais, financeiro e impostos, há uma margem líquida que representa o quanto esse lucro é em relação a margem líquida.  
Se a margem líquida for de 15%, isso quer dizer que para cada **100,00** que a empresa ganha, ela tem um lucro líquido de **R$ 15,00**.  

Mragem Líquida = Lucro Líquido / Receita Operacional Líquida x 100 (para encontrar em %)

In [15]:
df_ml = df_pl[df_pl['Mrg. Líq.'] > 10]

## Dividendos

Selelção das empresas pagadoras de dividendos acima de 6%.

In [16]:
df_div_yield = df_ml[df_ml['Div.Yield'] > 6]
df_div_yield.sort_values('Div.Yield', ascending=False)

,Papel,Cotação,Preco Justo,Div.Yield,P/L,P/VP,PSR,P/Ativo,P/Cap.Giro,P/EBIT,...,EV/EBITDA,Mrg Ebit,Mrg. Líq.,Liq. Corr.,ROIC,ROE,Liq.2meses,Patrim. Líq,Dív.Brut/ Patrim.,Cresc. Rec.5a
379,TRPN3,0.71,12.40,104.74,0.44,0.54,0.263,0.399,0.59,0.36,...,-0.35,72.28,59.29,3.61,523.01,122.35,0.000000e+00,5.935800e+09,0.00,5.77
441,WHRL4,6.21,27.33,26.40,5.26,3.09,1.205,1.222,5.75,25.37,...,15.98,4.75,22.94,1.38,10.42,58.79,6.409730e+07,3.018610e+11,0.20,-7.53
443,WHRL3,6.26,24.84,23.80,5.30,3.12,1.215,1.232,5.80,25.58,...,16.12,4.75,22.94,1.38,10.42,58.79,1.297650e+07,3.018610e+11,0.20,-7.53
654,SEER3,15.20,38.46,15.18,14.35,1.50,1.533,0.765,8.22,7.66,...,4.74,20.02,10.69,1.59,11.51,10.42,1.623840e+09,1.308490e+11,0.14,5.78
417,CPLE5,27.51,65.81,14.35,3.78,0.44,0.463,0.196,2.94,2.12,...,3.48,21.81,12.70,1.48,10.52,11.53,0.000000e+00,1.725240e+12,0.67,3.32
635,CYRE3,13.34,30.35,13.65,12.82,1.12,1.357,0.534,1.69,12.11,...,13.67,11.20,13.59,2.44,5.19,8.73,1.560830e+10,4.762380e+11,0.53,-2.12
601,ENAT3,9.00,19.05,12.70,11.10,0.83,2.152,0.526,1.59,7.75,...,1.27,27.77,19.38,3.63,11.35,7.44,1.887960e+09,2.895540e+11,0.09,23.71
505,ROMI3,9.78,19.17,11.76,7.39,0.77,0.758,0.399,1.71,12.59,...,10.89,6.02,10.29,1.70,3.80,10.47,2.832340e+08,7.944080e+10,0.57,7.72
656,UCAS3,3.92,7.38,11.30,14.40,1.62,1.735,1.135,3.98,15.06,...,7.41,11.52,12.04,2.10,10.75,11.24,1.329280e+08,1.600420e+10,0.00,-9.64
486,ITSA4,8.29,14.50,10.49,6.76,1.26,13.923,1.033,18.32,372.87,...,77.00,3.73,211.04,2.48,0.30,18.67,3.471680e+10,5.523200e+12,0.08,1.88


Vamos plotar o hitórico de Itausa para ver a variação de preço no utlimos 5 anos..

In [29]:
price = pd.DataFrame()

tickers = ['ITSA4.SA']
for t in tickers:
    price[t]=wb.DataReader(t, data_source='yahoo', start='2015-1-1')['Adj Close']
    
price.iplot()